In [5]:
import numpy as np
import random as rand

def check_optimality(gk,Hk,tolT): 
#   Revisa que se cumplan las condiciones de optimalidad: Grad=0, Hess positiva semidefinida

#   gk - Vector Gradiente de f en xk
#   Hk - Matriz Hessiana de f en xk
#   tolT - Tolerancia de el gradiente
    print(gk)
    return es_pos_def(Hk) and all(abs(gk)<tolT)
def Grad(f, xk, h):
#   Calcula el gradiende de f en xk, con diferencia h

#   f - Función
#   xk - Punto a evaluar
#   h - Diferencia de aproximación

    n = xk.size
    g = np.zeros(n)
    for i in range(0, n):
        b = np.copy(xk)
        b[i] += h
        g[i] = (f(b) - f(xk))/(h)
    return g

def Hess(f,xk,h):
#   Calcula el Hessiano en xk, con diferencia h

#   f - Función
#   xk - Punto a evaluar
#   h - Diferencia de aproximación

    n=xk.size
    H=np.zeros((n,n))
    for i in range(0,n):
        for j in range(0,n):
            ff=np.copy(xk)
            ff[i]+=h
            ff[j]+=h
            
            fb=np.copy(xk)
            fb[i]+=h
            fb[j]-=h
            
            bf=np.copy(xk)
            bf[i]-=h
            bf[j]+=h
            
            bb=np.copy(xk)
            bb[i]-=h
            bb[j]-=h
            
            H[i,j]=(f(ff)-f(fb)-f(bf)+f(bb))/(4*h**2)
    return H

def es_pos_def(A):
#   Prueba si la matriz A es positiva semidefinida si A es simétrica

#   A - Matriz a probar

        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
        return False
def PruebaMod_Hess(Hk, Beta ) :
#   Modifica la Hessiana para que sea positiva semidefinida, según el algoritmo 3.3 de Nocedal Ed. 2

#   Hk - Matrizz hessiana a modificar
#   Beta - Valor arbitrario a aumentar

    n=int(np.sqrt(np.size(Hk)))
    tk=0
    v=np.matrix.diagonal(Hk)
    
    if min(v)<0:
        tk=-min(v)+Beta    
    

    while es_pos_def(Hk+np.eye(n)*tk)==False:
           tk= max(2*tk, Beta);
    
    return Hk+np.eye(n)*tk
     
def BactrackSearch(f,xk,pk,gk,alpha0,c,rho):
#   Busca el paso según el algoritmo de 3.1 de Nocedal Ed. 2

#   f - Función a evaluar
#   xk - punto alrededor del que se evalúa
#   pk -Dirección elegida previamente
#   alpha0 - Valor de paso máximo
#   c - peso sobre la derivada direccional
#   rho - factor de disminución del paso

    alpha_k=alpha0
   

    while f(xk+alpha_k*pk)> f(xk)+c*alpha_k*np.dot(gk,pk):
        
        alpha_k=alpha_k*rho
    return alpha_k

def fi(f,xk,pk):
    return lambda a: f(xk + a*pk)

def Inter(l, b):
    m = (l+b)/2
    return m

def Zoom(alo, ahi, f, xk, pk, c1, c2,  h, maxiter):
    
    a =np.zeros(maxiter+1)
    fi_gorro = fi(f, xk, pk)
    fi0 = fi_gorro(0)
    fid0 = Grad(fi_gorro, np.zeros(1), h)
    ac = 0
    found = False


    for i in range(0,maxiter):
        
        a[i] = Inter(alo, ahi)
        fid = Grad(fi_gorro, np.array([ a[i] ]), h)
        fie = fi_gorro(a[i])    
    
        if (fie>(fi0 + c1*a[i]*fid0)) or (fie>=fi_gorro(alo)):
            ahi = a[i]    
            
        else:
            if abs(fid)<= -c2*fid0:
                ac = a[i]
                found = True
                break
            if fid*(ahi-alo)>= 0:
                ahi = alo
            alo = a[i]

    if not found:
        print("No convergimos en la interpolacion con {} max iteraciones".format(maxiter))
        ac = a[i]

    return ac  

def LineSearchWolf(f,a_max, xk, pk, c1, c2, h, maxiter):
    
    
    a =np.zeros(maxiter+1)
    fi_gorro = fi(f,xk,pk)
    fi0 = fi_gorro(0)
    fid0 = Grad(fi_gorro, np.zeros(1), h)
    for i in range(0,maxiter):
        
        a[i]= rand.uniform(a[i-1], a_max)
        
        fie = fi_gorro(a[i])
        fid = Grad(fi_gorro, np.array([ a[i] ]), h)
        
        
        if (fie > (fi0 + c1*a[i]*fid0)) or (fie >= fi_gorro(a[i-1])and (i>1)):
            a[i+1] = Zoom(a[i-1],a[i], f, xk, pk,c1, c2, h, maxiter)
            break
        
        if abs(fid) <= -c2*fid0:
            a[i+1] = a[i] 
            break
                    
        if fid >= 0:
            a[i+1] = Zoom(a[i],a[i-1], f, xk, pk, c1, c2, h, maxiter)
            break
    return a[i+1]
    
    
    
    
    
    
class Optimaizer3000:
    def __init__(self,f,x0):
        self.f=f
        self.x0=x0

   
 #----------------------------------------------------------------------------------------------------------------------------   
    
    
    
    
    def Min_LineSearchNewtonHessMod(self, tolT , h1 ,h2 , alpha0, c, rho,maxit):  
        x0=self.x0
        f=self.f
    #   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de búsqueda lineal de Newton
    #   con una aproximación de la Hessiana adaptada para garantizar que sea positiva definida.

    #   f-Función
    #   x0 - Punto inicial
    #   tolT -  Tolerancia sobre condición de optimalidad
    #   h1 -Diferencias para gradiente
    #   h2 -Diferencias para Hessiana
    #   c - peso sobre el gradiente para búsqueda lineal Bactracking
    #   rho - factor de disminución del paso para búsqueda lineal Bactracking
    #   maxit - Máximo número de Iteraciones

        xk=x0
        n=np.size(x0)
        for k in range(0,maxit):
    
    
            gk=Grad(f,xk,h1)                                       
            Hk=Hess(f,xk,h2)         
        
            if check_optimality(gk,Hk,tolT) :
                break
        
            HessMod=PruebaMod_Hess( Hk, 1e-3 ) 

            pk= -np.linalg.solve(HessMod,gk)
        
        
            alpha_k=BactrackSearch(f,xk, pk, gk, alpha0 ,c,  rho) 
        
            xk_1=xk
            xk=xk+alpha_k*pk
        
        
            print(xk)
            print(f(xk))
            
        return [f(xk),xk,k]
    
    
 #-----------------------------------------------------------------------------------------------------------------------------   
    
   
    def Min_LineSearchNewton(self , tolT , h1 ,h2 , alpha0, c, rho,maxit):  
        
    #   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de búsqueda lineal de Newton
    #   con una aproximación de la Hessiana adaptada

    #   f-Función
    #   x0 - Punto inicial
    #   tolT -  Tolerancia sobre condición de optimalidad
    #   h1 -Diferencias para gradiente
    #   h2 -Diferencias para Hessiana
    #   c - peso sobre el gradiente para búsqueda lineal Bactracking
    #   rho - factor de disminución del paso para búsqueda lineal Bactracking
    #   maxit - Máximo número de Iteraciones

        x0=self.x0
        xk=x0
        f=self.f
        n=np.size(x0)
        for k in range(0,maxit):
    
    
            gk=Grad(f,xk,h1)                                       
            Hk=Hess(f,xk,h2)         
        
            if check_optimality(gk,Hk,tolT) :
                break
        
            pk= -np.linalg.solve(Hk,gk)
        
        
            alpha_k=BactrackSearch(f,xk, pk, gk, alpha0 ,c,  rho) 
        
            xk_1=xk
            xk=xk+alpha_k*pk
        
        
            print(xk)
            print(f(xk))
            
        return [f(xk),xk,k]
    
    
 #--------------------------------------------------------------------------------------------------------------------------   
    
    def BFGS(self,epsilon,maxit,h1,h2):
        x0=self.x0
        f=self.f
    #Busca el mínimo de la función con un método Quasinewton y una matriz aproximada con BFGS 
    #   f- Función
    #   x0- Punto inicial
    #   epsilon- Tolerancia para la condición del gradiente
    #   h1- diferencia para la primera aproximación de Hessiana
    #   h2- diferencia para la aproximación del Gradiente

        n=np.size(x0)
        Hk=np.linalg.inv(Hess(f,x0,h1))
        gk=Grad(f,x0,h2)
        print(gk)
        xk=x0
        for k in range(0 ,maxit):
            if all(abs(gk)<epsilon):
                break
            pk=np.matmul(-Hk,gk)
            
            #alphak=BactrackSearch(f,xk,pk,gk,1,0.8,0.9)
            alphak= LineSearchWolf(f,1, xk, pk, 0.8, 0.9, 0.00001, 100)
            
            xk_1=alphak*pk+xk
            gk_1=Grad(f,xk_1,h2)
            sk= xk_1-xk
            yk= gk_1-gk
        
            rhok=1/np.dot(yk,sk)
            print(rhok)    
            Hk= np.matmul(np.matmul((np.identity(n)-rhok*np.matmul(sk,np.transpose(yk))),Hk),(np.identity(n)-rhok*np.matmul(yk,np.transpose(sk))))
            gk=gk_1
            xk=xk_1
            print(xk)
            print(f(xk))

        return [f(xk),xk,k]
#--------------------------------------------------------------------------------------------------------------------------    

    def NewtonAlgorithm(self,tolT , h1 ,h2 ,maxit):  
        x0=self.x0
        f=self.f
    #   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de Newton sin ninguna modificación o 
    #   procedimiento extra.

    #   f-Función
    #   x0 - Punto inicial
    #   tolT -  Tolerancia sobre condición de optimalidad
    #   h1 -Diferencias para gradiente
    #   h2 -Diferencias para Hessiana
    #   c - peso sobre el gradiente para búsqueda lineal Bactracking
    #   rho - factor de disminución del paso para búsqueda lineal Bactracking
    #   maxit - Máximo número de Iteraciones

        xk=x0
        n=np.size(x0)
        for k in range(0,maxit):
    
    
            gk=Grad(f,xk,h1)                                       
            Hk=Hess(f,xk,h2)         
        
            if check_optimality(gk,Hk,tolT) :
                break
        
            pk= -np.linalg.solve(Hk,gk)
        
        
            xk_1=xk
            xk=xk+pk
        
        
            print(xk)
            print(f(xk))
            
        return [f(xk),xk,k]

In [29]:
N=10    
M=150  

def rosenbrock1_100(v):
    # a=1 b=100
    return ((1-v[0])**2+100*(v[1]-v[0]**2)**2)

def costo_camaras(c):
    
    
    global data
    global N
    global M
#Calcula el Costo de posicionar un número N de cámaras para cubrir M crimen. Hay uncosto cuadrático para la distancia 
#entre una cámara y un crimen. Y también hay un costo inverso a la distancia al cuadrado entre cámaras. Lo que implica
#una serie de singularidades para todos los puntos donde dos cámaras son iguales, por lo que esta no puede ser condición 
#inicial.

    costo=0

#   c-Lista de arrays, que indican la localización de las cámaras,   N arrays
#   data-Lista de arrays, que indican la localicación de los crímenes, M arrays
#   delta= peso de distancia camaras crimenes
#   sigma= peso del inverso de la distancia camara camara
    
    #Primero el costo de distancia de cámaras a crímenes
    for i in range(0,2*N,2):
        for j in range(0,M-1):
            costo=costo+(c[i]-data[j][0])**2+(c[i+1]-data[j][1])**2
   
    #Luego el costo de cercanía de las cámaras
    for i in range(0,2*N,2):
        for j in range(0,2*N,2):
            if i!=j:
                costo=costo+1/((c[i]-c[j])**2+(c[i+1]-c[j+1])**2)
    
    #Regresamos finalmente el costo total:
    
    return costo

import csv
#En este script obtenemos los datos de nuestra función

with open('crime.csv', 'r') as file:
    reader = csv.reader(file)
    n=31057

    rows=[]
    data=[]
    for row in reader: 
        rows.append(row)

for i in range(1,M):
    data.append(np.asarray(rows[i][3:],dtype=float))

  

#N=800    
#M=1305

#N=8000
#M=31057

c0=np.zeros(2*N)
for i in range(0,2*N):
    c0[i]=rand.uniform(0, 100)
c0=np.array(c0)
    
#-----------------------------------------------------------------------------------------------------------------------------    
    

In [30]:
Camaritas=Optimaizer3000(costo_camaras,c0)
Optimaizer3000.NewtonAlgorithm(Camaritas,0.0000000001 , 0.00000001,0.00000001 ,100)

[ 1057.98244476 43391.43633842 23961.81225777 57202.57759094
  4425.64487457 34216.79139137 -4357.84459114 50261.61670685
 13596.56453133 32924.11565781 -5463.51075172 39806.96201324
  9888.41056824 31881.03437424 14899.67107773 42526.42393112
 -2729.89273071 59208.27388763 10814.51773643 30927.36005783]


LinAlgError: Singular matrix

In [31]:
Optimaizer3000.Min_LineSearchNewtonHessMod(Camaritas, 0.0000001 , 0.00001 ,0.00001 , 1, 0.9, 0.9,100)

[ 1058.26854706 43392.04058051 23962.7122879  57203.17065716
  4425.42806268 34217.10729599 -4357.23364353 50260.80459356
 13596.12867236 32925.07305741 -5463.25668693 39807.12071061
  9886.4197731  31880.89579344 14896.18048072 42526.76442266
 -2730.40011525 59207.75681734 10816.62923098 30930.41703105]
[ 18.08843678  13.02813213  79.51353578  54.52450535  33.50887148
   0.65211432   3.75112609  45.02711686  60.41110697  -9.57741721
   7.31575153  -2.83193273  47.40571637  -8.91634342  63.19649228
  23.23456428  11.76645721  72.59232561  44.42055701 -20.33694342]
22696150.895532254
[ -386.07493043 33425.06289482 17918.60461235 45790.98336399
  4209.21482146 29737.00985312 -4658.59137475 42960.76111495
 12226.07791424 26688.6100173  -3596.33527696 28698.76474142
  8350.47587752 26885.61044633 13056.12400174 36466.57988429
 -2270.02389729 51175.19460618  7460.89406312 23482.27538168]
[ 19.4210199   -7.24595211  68.65405731  28.1284675   32.02491502
 -14.55959419   3.97087125  22.8565540

[  62.02840304 1849.97015749 1041.12309637 2699.72120877  231.55566887
 1742.45568633 -229.59877388 2548.99757711  727.74307919 1445.24741772
 -139.07535758 1605.09461712  470.59456556 1627.51328753  643.38604425
 1908.62105374 -145.63412115 2967.05133951  411.29324818 1433.77955537]
[ 19.51148252 -94.03490863  22.23134057 -91.7530344   20.02720895
 -94.378447    18.75348993 -92.17103514  21.40666425 -95.20149995
  18.97966552 -94.76405188  20.69713389 -94.66307491  21.16105526
 -93.89492134  18.98672504 -91.01697254  20.48780709 -95.27663455]
49830.632098447604
[  51.91448363 1505.52293635  847.91337722 2199.61857219  189.02077936
 1419.42944319 -186.69885394 2076.92236072  591.40794328 1178.20886881
 -112.29826268 1308.51138892  382.1022372  1325.54615848  523.35941436
 1553.98467323 -118.60414088 2416.7010888   335.88061706 1170.29998437]
[ 19.46191396 -94.95059897  21.70530958 -93.1198578   19.9072868
 -95.26141436  18.86763077 -93.46459598  21.05670241 -95.94341626
  19.03638411 -

[  6.40526994  89.09371619  38.62803334 118.01581978  12.03255154
  77.62863368  -5.50966648 111.45819103  31.56545871  72.35707277
  -2.80189001  78.25614359  15.86835034  76.40627275  24.04840884
  88.10300297   0.61743874 123.6751413   19.30149472  69.28413732]
[ 19.3972642  -98.69171851  19.73375607 -98.44737768  19.23989585
 -99.03919354  18.96250567 -98.67350581  19.8290787  -99.30606308
  18.89268804 -99.14958523  19.6025699  -99.01151413  19.93429671
 -98.82315035  19.26017717 -98.33011274  19.37064695 -99.42138624]
741.1776584153525
[  4.26815276  72.83827194  29.61399134  94.91634013  14.81667788
  66.35268641  -2.79576328  88.68139308  27.23131895  59.48864901
  -9.56920699  62.44971808  15.11954664  63.71559147  19.41559069
  72.16899505   1.85591783 101.39152217  14.20062514  54.53886314]
[ 19.39182743 -98.735165    19.7248682  -98.49844353  19.23518499
 -99.07927786  18.9561494  -98.72451606  19.81099541 -99.35351242
  18.89185383 -99.19718697  19.59812576 -99.05044718  1

[ 0.21445961  3.38211971  1.3361373   4.36386192  0.66808047  3.06707715
 -0.12235381  4.09354593  1.28139313  2.67481747 -0.43352612  2.88507903
  0.6334505   3.00129559  0.92373282  3.36952285  0.10968731  4.67272365
  0.66721187  2.54309872]
[ 19.36267188 -98.92053046  19.68689493 -98.71077975  19.22228035
 -99.25809052  18.9297393  -98.93909974  19.73419215 -99.5458105
  18.88636268 -99.3872091   19.58505556 -99.21147631  19.92448066
 -99.07259145  19.21379936 -98.59190976  19.30874624 -99.63464529]
646.002351507956
[ 0.17480161  2.75571645  1.0883984   3.55495167  0.54394453  2.49861649
 -0.09971269  3.33488321  1.04400937  2.1784015  -0.35312904  2.35056415
  0.51574794  2.44559898  0.75277088  2.74544722  0.08951181  3.80675503
  0.54388626  2.0721387 ]
[ 19.3623419  -98.92223124  19.68653395 -98.7126816   19.22224979
 -99.2598043   18.92952362 -98.94102397  19.73346403 -99.54748132
  18.88629046 -99.38881021  19.58499667 -99.21287796  19.92443381
 -99.0743939   19.21346161 -98.

[ 0.01045734  0.1647928   0.06509366  0.21250668  0.03243923  0.14938296
 -0.00604438  0.19932053  0.06251507  0.12985224 -0.02121697  0.14046443
  0.03068711  0.14637114  0.04520365  0.16425122  0.00531284  0.22761028
  0.03261968  0.12378849]
[ 19.36138597 -98.92921644  19.6860441  -98.72130236  19.22186907
 -99.26657003  18.92911446 -98.94779768  19.72938027 -99.55518075
  18.88533222 -99.39407675  19.58459149 -99.21906307  19.9244233
 -99.08316351  19.21346143 -98.60108508  19.30493622 -99.6429722 ]
645.8680925589997
[ 0.00904137  0.14254183  0.05632874  0.18383757  0.02805045  0.12920341
 -0.0052679   0.17240917  0.05410549  0.11226232 -0.01839741  0.12146405
  0.02654374  0.12660463  0.03915252  0.14207545  0.00457408  0.19691074
  0.0282106   0.10701455]
[ 19.36145557 -98.9292657   19.68619683 -98.72148807  19.22181702
 -99.2665652   18.92918248 -98.94768105  19.72918755 -99.55537458
  18.88522652 -99.39393061  19.58455705 -99.21918935  19.92444357
 -99.08344047  19.21366452 -98

[ 0.00420122  0.06657377  0.02642879  0.08595078  0.01302575  0.06021658
 -0.00267989  0.08050529  0.02540983  0.05213859 -0.00882536  0.05651491
  0.0124436   0.05904369  0.01854542  0.06630569  0.0020337   0.09211423
  0.01312492  0.04965318]
[ 19.36208277 -98.92939134  19.68749402 -98.7226887   19.22139492
 -99.2662418   18.92977014 -98.94643093  19.72774315 -99.55668016
  18.8843856  -99.39249602  19.58428235 -99.21999644  19.9246077
 -99.08539789  19.21536038 -98.60074717  19.30301842 -99.64293172]
645.8678231611995
[ 0.0041897   0.06639154  0.02635655  0.08571544  0.01299011  0.06005154
 -0.00267269  0.08028485  0.02534026  0.05199573 -0.00880125  0.0563601
  0.01240946  0.05888201  0.01849477  0.06612404  0.00202803  0.09186213
  0.01308897  0.04951703]
[ 19.3620865  -98.92939148  19.68750158 -98.72269493  19.2213925
 -99.26623933  18.92977359 -98.94642312  19.72773512 -99.55668707
  18.88438082 -99.39248726  19.58428078 -99.22000061  19.92460863
 -99.0854085   19.21537021 -98.6

[ 0.00411787  0.06525737  0.02590665  0.08425091  0.01276837  0.05902474
 -0.00262759  0.07891308  0.02490734  0.05110697 -0.00865107  0.05539617
  0.01219738  0.05787595  0.01817956  0.06499389  0.00199278  0.09029322
  0.01286514  0.04867028]
[ 19.36210801 -98.92939237  19.68754515 -98.72273103  19.22137856
 -99.2662252   18.92979352 -98.94637815  19.72768873 -99.55672701
  18.88435327 -99.3924368   19.58427173 -99.2200247   19.92461398
 -99.08546978  19.2154269  -98.60073017  19.30295369 -99.64292522]
645.8678207134111
[ 0.00411784  0.06525713  0.02590654  0.08425061  0.01276833  0.05902452
 -0.00262752  0.07891286  0.02490727  0.05110679 -0.00865103  0.05539601
  0.01219735  0.05787573  0.01817948  0.0649937   0.00199283  0.09029291
  0.01286512  0.04867007]
[ 19.36210801 -98.92939237  19.68754515 -98.72273103  19.22137856
 -99.2662252   18.92979352 -98.94637815  19.72768873 -99.55672702
  18.88435326 -99.39243679  19.58427173 -99.2200247   19.92461398
 -99.08546979  19.21542691 -9

[645.8678207130437,
 array([ 19.36210801, -98.92939237,  19.68754515, -98.72273103,
         19.22137856, -99.2662252 ,  18.92979352, -98.94637814,
         19.72768872, -99.55672702,  18.88435326, -99.39243679,
         19.58427172, -99.2200247 ,  19.92461398, -99.08546979,
         19.21542691, -98.60073017,  19.30295368, -99.64292522]),
 99]

In [32]:
Optimaizer3000.Min_LineSearchNewton(Camaritas , 0.00000001, 0.000001,0.000001 , 1, 0.8, 0.9,100)

[ 1058.26556683 43392.03983545 23962.71377802 57203.16618681
  4425.43625832 34217.11176634 -4357.24109411 50260.81204414
 13596.12494707 32925.09168386 -5463.26488256 39807.11102486
  9886.43616438 31880.87791204 14896.18420601 42526.76665783
 -2730.39937019 59207.79705048 10816.59644842 30930.39244413]
[ 29.73432862  14.06240276  99.57613494  89.73380228  29.49568083
  33.79785609   4.63026425  66.98358638  60.88950388  23.30920964
 -10.50353662  29.02955003  37.6028177    2.46215963  79.70267538
   3.05698041   6.58177605  81.76222249  64.68948768  12.74385266]
30819296.747505277
[ 3084.40625668 33733.27851295 23897.25297689 56283.35475922
  3013.27556372 39614.41665888 -4396.60996199 49503.80325317
 12368.64924431 36488.82359266 -8906.47619963 38193.4531033
  5429.22317982 30276.40283108 17974.99507666 30453.66704464
 -3815.06979465 53907.83771873 13501.02201104 33340.37214518]
[ 29.78219529  19.33234721  88.97463289  85.95012527  35.76180253
  34.43629347   4.98454942  68.00505014

LinAlgError: Singular matrix

In [35]:
Optimaizer3000.BFGS(Camaritas,0.00000001,100,0.000001,0.000001)

[ 1058.26556683 43392.03983545 23962.71377802 57203.16618681
  4425.43625832 34217.11176634 -4357.24109411 50260.81204414
 13596.12494707 32925.09168386 -5463.26488256 39807.11102486
  9886.43616438 31880.87791204 14896.18420601 42526.76665783
 -2730.39937019 59207.79705048 10816.59644842 30930.39244413]
2.5774885985707382e-06
[26.69261664 28.56253361 99.67439685 91.11467097 31.61564887 25.69515992
  4.68937795 68.12004821 62.73226518 17.95916156 -5.33442754 31.45200113
 44.29411072  4.8709387  75.08066864 21.18161456  8.2101133  89.71869518
 60.65959413  9.12591544]
31947520.48991477
6.125313508893623e-07
[  7.33781425  17.88271173  83.4268746   74.79248889  16.32298634
   4.382829   -12.70836715  51.50414702  46.69088222  -2.19305803
 -20.70633083  15.56075998  31.1564113  -10.87534111  55.68204124
  13.68346768  -8.35181331  76.81208863  41.27980781 -10.12796705]
25277802.33010509
1.7180031862437721e-07
[-23.60356238 -13.93222436  52.18892092  43.55516898 -15.01555691
 -26.35210462 

<ipython-input-5-de92eab35e95>:310: RuntimeWarning: divide by zero encountered in double_scalars
  rhok=1/np.dot(yk,sk)
<ipython-input-5-de92eab35e95>:312: RuntimeWarning: invalid value encountered in double_scalars
  Hk= np.matmul(np.matmul((np.identity(n)-rhok*np.matmul(sk,np.transpose(yk))),Hk),(np.identity(n)-rhok*np.matmul(yk,np.transpose(sk))))


nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan
nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
nan

KeyboardInterrupt: 

In [149]:
Rosenbrock=Optimaizer3000(rosenbrock1_100,np.array([0.799999448753295, 5.450318174756265]))
print(Rosenbrock)

Optimaizer3000.NewtonAlgorithm(Rosenbrock,0.0000000001 , 0.0000001,0.0000001 ,100)

[-1539.70110659   962.06382295]
[0.82293267 0.51020649]
2.8206432079780153
[ 54.62164518 -33.40232743]
[0.8280573 0.6856561]
0.029564344705810208
[-0.33630828 -0.00454862]
[0.99919177 0.96909712]
0.08577394419920122
[11.70378695 -5.8574055 ]
[0.99930366 0.99860772]
4.848958253062731e-07
[-1.32244910e-03 -5.10801859e-06]
[0.99996997 0.99993946]
9.259212206043262e-10
[ 1.77633485e-04 -8.87962169e-05]
[0.99996995 0.99993985]
9.031625551362875e-10
[-6.50355247e-12 -3.33064274e-14]


[9.031625551362875e-10, array([0.99996995, 0.99993985]), 6]

In [119]:
Optimaizer3000.BFGS(Rosenbrock,0.00000001,20,0.0000001,0.0000001)

[-1539.70110659   962.06382295]
0.0038741266522512533
[0.80523474 4.32256813]
1349.9868813964183
0.003768085189103711
[1.67967717 4.21615985]
195.0210668062314
0.02371975745103925
[1.56652642 3.35883959]
82.19351002331027
0.13126312091683495
[1.62912866 3.34837312]
48.60284626129392
inf
[1.62912866 3.34837312]
48.60284626129392
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan
nan
[nan nan]
nan


<ipython-input-115-1c676a756194>:310: RuntimeWarning: divide by zero encountered in double_scalars
  rhok=1/np.dot(yk,sk)
<ipython-input-115-1c676a756194>:312: RuntimeWarning: invalid value encountered in double_scalars
  Hk= np.matmul(np.matmul((np.identity(n)-rhok*np.matmul(sk,np.transpose(yk))),Hk),(np.identity(n)-rhok*np.matmul(yk,np.transpose(sk))))


[nan, array([nan, nan]), 19]

In [112]:
Optimaizer3000.Min_LineSearchNewton(Rosenbrock, 0.000001, 0.000001,0.000001 , 1, 0.9, 0.9,100)

[-1539.70174415   962.06391117]
[0.79999257 4.55883447]
1535.7756766334699
[-1254.41973387   783.76937086]
[0.79984108 3.83274442]
1019.5641124471894
[-1021.95730312   638.59983356]
[0.79972798 3.24107304]
676.8245986862624
[-832.60036922  520.30173936]
[0.7996859  2.75888099]
449.2187500080651
[-678.33722136  423.87679173]
[0.79959047 2.36603716]
298.1867735086783
[-552.65957133  345.3385483 ]
[0.79948349 2.04591213]
197.93146786201302
[-450.26666945  281.34775755]
[0.79935059 1.78502349]
131.38609943137786
[-366.84344568  229.21252483]
[0.79919422 1.57239447]
87.2167305887814
[-298.87918656  186.7367141 ]
[0.79899354 1.39906027]
57.902226870934925
[-243.50994567  152.134019  ]
[0.79874561 1.25771165]
38.44543137271033
[-198.40090197  123.94351942]
[0.79844156 1.14239203]
25.531321815360716
[-161.65082585  100.97672305]
[0.79806645 1.04823792]
16.95983859817449
[-131.71048125   82.26567344]
[0.79760519 0.97128206]
11.270702713807806
[-107.31816156   67.02170563]
[0.79703798 0.90828082

[0.09749407418777994, array([0.73462691, 0.55613002]), 99]

In [113]:
Optimaizer3000.Min_LineSearchNewtonHessMod(Rosenbrock, 0.000001, 0.000001,0.000001 , 1, 0.9, 0.9,100)

[-1539.70174415   962.06391117]
[1.84460236 5.24248789]
339.2475995710627
[-1355.8855029   367.9861033]
[2.8289581  7.96442812]
3.4938971692270666
[47.31288334 -7.71506502]
[2.78571832 7.72772353]
3.2944347013708457
[39.79227602 -6.50050839]
[2.73666692 7.46112892]
3.0956314945945453
[34.36446808 -5.64328532]
[2.68279345 7.17207095]
2.8958519790534054
[30.52877339 -5.06184421]
[2.62560691 6.87044275]
2.6972082872374394
[27.7969773  -4.67367661]
[2.56656246 6.56519908]
2.5027107018023758
[25.76644767 -4.4086535 ]
[2.50688458 6.26340377]
2.3150809517677065
[24.14081998 -4.21320364]
[2.44731856 5.96909084]
2.1358478142578825
[22.74702312 -4.05535595]
[2.38834065 5.68459041]
1.9658300369558888
[21.48509259 -3.91603413]
[2.33013964 5.41061413]
1.8051310255429074
[20.3124437  -3.78722405]
[2.27286436 5.14759395]
1.6537402197565372
[19.20193597 -3.66358898]
[2.21660806 4.89563952]
1.5115058368617398
[18.13920437 -3.54225344]
[2.16140875 4.65457561]
1.3781529540984754
[17.11925629 -3.42233587]

[6.864944491046329e-06, array([1.00251791, 1.0049697 ]), 99]